In [47]:
!pip install git+https://github.com/keras-team/keras-tuner.git
!pip install autokeras
from IPython.display import clear_output
clear_output(wait=False)


In [99]:
import pandas as pd
import os, pickle
import numpy as np
import autokeras as ak

In [105]:
path = "./datasets/DEAP/deap-dataset/data_preprocessed_python/"
data = np.array([[]])
labels = np.array([[]])
test_data = np.array([[]])
test_labels = np.array([[]])

i = 0
for files in os.listdir(path):
    file = os.path.join(path, files)
    
    print(file)
    with open(file, "rb") as f:
        content = pickle.load(f, encoding="latin1")
        if i==0:
            if file.endswith("s28.dat") or file.endswith("s14.dat"):
                test_data = content['data']
                test_labels = content['labels']
                continue
            print("first")
            labels = content['labels']
            data = content['data']
            i=i+1
            print(i)
            continue
        print("next")
        if file.endswith("s28.dat") or file.endswith("s14.dat"):
                test_data = np.concatenate((test_data, np.array(content['labels'])))
                test_labels = np.concatenate((test_data, np.array(content['data'])))
                continue
        labels = np.concatenate((labels, np.array(content['labels'])))
        data = np.concatenate((data, np.array(content['data'])))


./datasets/DEAP/deap-dataset/data_preprocessed_python/s28.dat
./datasets/DEAP/deap-dataset/data_preprocessed_python/s14.dat
./datasets/DEAP/deap-dataset/data_preprocessed_python/s01.dat
first
1
./datasets/DEAP/deap-dataset/data_preprocessed_python/s15.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s29.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s03.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s17.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s16.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s02.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s06.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s12.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s13.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s07.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s11.dat
next
./datasets/DEAP/deap-dataset/data_preprocessed_python/s05.dat
next
./

In [80]:
content['data'].size

12902400

In [81]:
content['labels'].size

160

In [106]:
data.shape

(1200, 40, 8064)

In [107]:
labels.shape

(1200, 4)

In [108]:
test_data.shape

(40, 40, 8064)

In [110]:
test_labels.shape

(40, 4)

In [97]:
valence = labels[:,0]
arousal = labels[:,1]
dominance = labels[:,2]
liking = labels[:,3]


In [111]:
test_valence = test_labels[:,0]
test_arousal = test_labels[:,1]

# LSTM

In [120]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import json
import matplotlib.pyplot as plt

In [137]:
# ----------------- Hyperparams -----------------
BATCH_SIZE = 32
EPOCHS = 200
VALIDATION_SPLIT = 0.2  # from X_train used as val
LEARNING_RATE = 1e-3
PATIENCE_ES = 15
PATIENCE_RLR = 5
INPUT_TIMESTEPS = 40
INPUT_FEATURES = 8064
TARGET_DIM = 1
# ----------------- Model builder -----------------
def build_lstm_model(timesteps=INPUT_TIMESTEPS, features=INPUT_FEATURES, output_dim=TARGET_DIM):
    inp = layers.Input(shape=(timesteps, features), name="eeg_input")

    # Optional small input projection to reduce dimensionality (improves speed)
    x = layers.TimeDistributed(layers.Dense(128, activation="relu"))(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # Stacked LSTMs with return_sequences for intermediate layers
    x = layers.Bidirectional(
        layers.LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Bidirectional(
        layers.LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)
    )(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Bidirectional(
        layers.LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)
    )(x)
    x = layers.BatchNormalization()(x)

    # Final LSTM returns last output (no return_sequences)
    x = layers.Bidirectional(
        layers.LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.1)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Dense head
    x = layers.Dense(64, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.25)(x)

    out = layers.Dense(output_dim, activation="linear", name="fuzzy_out")(x)

    model = models.Model(inputs=inp, outputs=out, name="deep_bi_lstm_regressor")
    return model


In [138]:
# ----------------- Reproducibility -----------------
import random
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [139]:
model = build_lstm_model()
model.summary()
OUT_DIR = "./"
# ----------------- Compile -----------------
opt = optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss="mae", metrics=["mae"])

# ----------------- Callbacks -----------------
checkpoint_path = os.path.join(OUT_DIR, "best_model.h5")
cb_early = callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE_ES, restore_best_weights=True, verbose=1)
cb_checkpoint = callbacks.ModelCheckpoint(checkpoint_path, monitor="val_loss", save_best_only=True, verbose=1)
cb_reduce = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=PATIENCE_RLR, verbose=1, min_lr=1e-6)

# Also save training logs
history_path = os.path.join(OUT_DIR, "history.json")

# ----------------- Fit -----------------
history = model.fit(
    data, valence,
    validation_split = 0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[cb_early, cb_checkpoint, cb_reduce],
    verbose=2,
    shuffle=True
)

Model: "deep_bi_lstm_regressor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ eeg_input (InputLayer)          │ (None, 40, 8064)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 40, 128)        │     1,032,320 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 40, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_19                │ (None, 40, 512)        │       788,480 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 40, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_20                │ (None, 40, 1024)       │     4,198,400 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 40, 1024)       │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_21                │ (None, 40, 512)        │     2,623,488 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 40, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_22                │ (None, 40, 256)        │       656,384 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 40, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_23                │ (None, 128)            │       164,352 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 9,482,241 (36.17 MB)

 Trainable params: 9,476,993 (36.15 MB)

 Non-trainable params: 5,248 (20.50 KB)

...

30/30 - 37s - 1s/step - loss: 2.6140 - mae: 2.6140 - val_loss: 2.0488 - val_mae: 2.0488 - learning_rate: 0.0010
Epoch 5/200

Epoch 5: val_loss improved from 2.04878 to 1.90193, saving model to ./best_model.h5


30/30 - 36s - 1s/step - loss: 2.4961 - mae: 2.4961 - val_loss: 1.9019 - val_mae: 1.9019 - learning_rate: 0.0010
Epoch 6/200

Epoch 6: val_loss improved from 1.90193 to 1.68398, saving model to ./best_model.h5


30/30 - 37s - 1s/step - loss: 2.2517 - mae: 2.2517 - val_loss: 1.6840 - val_mae: 1.6840 - learning_rate: 0.0010
Epoch 7/200

Epoch 7: val_loss improved from 1.68398 to 1.58867, saving model to ./best_model.h5


30/30 - 38s - 1s/step - loss: 2.1786 - mae: 2.1786 - val_loss: 1.5887 - val_mae: 1.5887 - learning_rate: 0.0010
Epoch 8/200

Epoch 8: val_loss did not improve from 1.58867
30/30 - 39s - 1s/step - loss: 2.0106 - mae: 2.0106 - val_loss: 1.8155 - val_mae: 1.8155 - learning_rate: 0.0010
Epoch 9/200

Epoch 9: val_loss did not improve from 1.58867
30/30 - 40s - 1s/step - loss: 1.9415 - mae: 1.9415 - val_loss: 1.9384 - val_mae: 1.9384 - learning_rate: 0.0010
Epoch 10/200

Epoch 10: val_loss did not improve from 1.58867
30/30 - 39s - 1s/step - loss: 1.9780 - mae: 1.9780 - val_loss: 1.8237 - val_mae: 1.8237 - learning_rate: 0.0010
Epoch 11/200

Epoch 11: val_loss did not improve from 1.58867
30/30 - 40s - 1s/step - loss: 1.9529 - mae: 1.9529 - val_loss: 1.8711 - val_mae: 1.8711 - learning_rate: 0.0010
Epoch 12/200

Epoch 12: val_loss did not improve from 1.58867

Epoch 12: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
30/30 - 40s - 1s/step - loss: 1.8796 - mae: 1.8796 - val

30/30 - 41s - 1s/step - loss: 1.8555 - mae: 1.8555 - val_loss: 1.4523 - val_mae: 1.4523 - learning_rate: 5.0000e-04
Epoch 17/200

Epoch 17: val_loss did not improve from 1.45228
30/30 - 43s - 1s/step - loss: 1.8842 - mae: 1.8842 - val_loss: 1.4677 - val_mae: 1.4677 - learning_rate: 5.0000e-04
Epoch 18/200

Epoch 18: val_loss did not improve from 1.45228
30/30 - 42s - 1s/step - loss: 1.8418 - mae: 1.8418 - val_loss: 1.4598 - val_mae: 1.4598 - learning_rate: 5.0000e-04
Epoch 19/200

Epoch 19: val_loss did not improve from 1.45228
30/30 - 37s - 1s/step - loss: 1.8559 - mae: 1.8559 - val_loss: 1.5175 - val_mae: 1.5175 - learning_rate: 5.0000e-04
Epoch 20/200

Epoch 20: val_loss did not improve from 1.45228
30/30 - 37s - 1s/step - loss: 1.8437 - mae: 1.8437 - val_loss: 1.5700 - val_mae: 1.5700 - learning_rate: 5.0000e-04
Epoch 21/200

Epoch 21: val_loss did not improve from 1.45228

Epoch 21: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
30/30 - 40s - 1s/step - loss: 1.

# AutoKeras


In [ ]:
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.Input()],
    outputs=[
        ak.RegressionHead(metrics=["mse"]),
        ak.RegressionHead(metrics=["mse"])
    ],
    overwrite=True,
    max_trials=5,
)
# Fit the model with prepared data.
model.fit(
    [data],
    [valence, arousal],
    epochs=100,
    batch_size=50,
    validation_split=0.2
)